In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import copy
from tqdm.notebook import tqdm_notebook
import math
from skopt import gbrt_minimize
from skopt.space import Real
import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

random_state = 1

mps


In [2]:
# read in data and split
X = torch.load('inputs/bfw_senet50_face_embeddings.pt').cpu()
y = torch.load('inputs/bfw_senet50_labels.pt').cpu()
df = pd.read_csv('inputs/bfw_senet50_df.csv')
gender = df['reference_gender']
df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
0,n004721,n004721,asian,asian,male,male,1.0
1,n004721,n004721,asian,asian,male,male,1.0
2,n004721,n004721,asian,asian,male,male,1.0
3,n004721,n004721,asian,asian,male,male,1.0
4,n004721,n004721,asian,asian,male,male,1.0
...,...,...,...,...,...,...,...
38395,n003412,n005685,white,white,male,male,0.0
38396,n008917,n008839,black,black,female,female,0.0
38397,n008452,n008150,asian,asian,male,male,0.0
38398,n004936,n002509,asian,asian,female,female,0.0


In [3]:
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)

train_X = X[train_split]
X_val = X[val_split]
test_X = X[test_split]

train_y = y[train_split]
y_val = y[val_split]
test_y = y[test_split]

train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]


train_gender = gender[train_split].values
train_gender[train_gender=='male'] = 0
train_gender[train_gender=='female'] = 1
train_gender = train_gender.astype(int)

gender_val = gender[val_split].values
gender_val[gender_val=='male'] = 0
gender_val[gender_val=='female'] = 1
gender_val = gender_val.astype(int)

test_gender = gender[test_split].values
test_gender[test_gender=='male'] = 0
test_gender[test_gender=='female'] = 1
test_gender = test_gender.astype(int)

In [4]:
train_matches = train_df[train_df.labels==1]
train_non_matches = train_df[train_df.labels==0]
print('non matches')
print(train_non_matches['reference_gender'].value_counts())
print('matches')
print(train_matches['reference_gender'].value_counts())

female_non_matches = train_non_matches[train_non_matches['reference_gender']=='female']
male_non_matches = train_non_matches[train_non_matches['reference_gender']=='male']

female_matches = train_matches[train_matches['reference_gender']=='female']
male_matches = train_matches[train_matches['reference_gender']=='male']

non matches
female    5804
male      5675
Name: reference_gender, dtype: int64
matches
female    5809
male      5752
Name: reference_gender, dtype: int64


In [5]:
np.random.seed(random_state)
female_matches_sub_idx = female_matches.index[np.random.choice(len(female_matches.index), size=5675, replace=False)]
male_matches_sub_idx = male_matches.index[np.random.choice(len(male_matches.index), size=5675, replace=False)]
female_non_matches_sub_idx = female_non_matches.index[np.random.choice(len(female_non_matches.index), size=5675, replace=False)]
male_non_matches_sub_idx = male_non_matches.index

X_train = torch.cat([X[female_matches_sub_idx],X[male_matches_sub_idx],X[female_non_matches_sub_idx],X[male_non_matches_sub_idx]])
y_train = torch.cat([y[female_matches_sub_idx],y[male_matches_sub_idx],y[female_non_matches_sub_idx],y[male_non_matches_sub_idx]])
gender_train = np.concatenate([gender[female_matches_sub_idx],gender[male_matches_sub_idx],gender[female_non_matches_sub_idx],gender[male_non_matches_sub_idx]])
df_train = pd.concat([df.iloc[female_matches_sub_idx],df.iloc[male_matches_sub_idx],df.iloc[female_non_matches_sub_idx],df.iloc[male_non_matches_sub_idx]])
gender_train[gender_train=='male'] = 0
gender_train[gender_train=='female'] = 1
gender_train = gender_train.astype(int)

In [6]:
test_split, holdout_split = train_test_split(np.arange(7680),test_size=0.1, random_state=random_state)
X_test = test_X[test_split]
X_holdout = test_X[holdout_split]
y_test = test_y[test_split]
y_holdout = test_y[holdout_split]
gender_test = test_gender[test_split]
gender_holdout = test_gender[holdout_split]
df_test = test_df.iloc[test_split]
df_holdout = test_df.iloc[holdout_split]
df_holdout

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
5873,n008307,n008307,indian,indian,male,male,1.0
24669,n000806,n005599,white,white,female,female,0.0
4525,n002800,n002800,black,black,male,male,1.0
22366,n002273,n000596,black,black,female,female,0.0
19758,n003655,n002638,black,black,female,female,0.0
...,...,...,...,...,...,...,...
4412,n001280,n001280,black,black,male,male,1.0
37015,n000541,n007272,white,white,male,male,0.0
28389,n007046,n001024,indian,indian,female,female,0.0
15345,n003573,n003573,indian,indian,female,female,1.0


In [7]:
## train data
class TrainData(data.Dataset):
    
    def __init__(self, X_data, y_data,gender):
        self.X_data = X_data
        self.y_data = y_data
        self.gender = gender    
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index], self.gender[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train),
                       gender_train)
test_data = TrainData(torch.FloatTensor(X_test),torch.FloatTensor(y_test),gender_test)
holdout_data = TrainData(torch.FloatTensor(X_holdout),torch.FloatTensor(y_holdout),gender_holdout)
val_data = TrainData(torch.FloatTensor(X_val), 
                       torch.FloatTensor(y_val),
                       gender_val)
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 4096
        self.layer_1 = nn.Linear(4096, 1024) 
        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(1024)
        self.layer_2 = nn.Linear(1024, 512)
        self.batchnorm2 = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(p=0.1)
        self.layer_out = nn.Linear(512, 1) 

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
    


In [8]:
EPOCHS = 20
BATCH_SIZE = 64
LEARNING_RATE = 1e-5
eps = 1e-5
model = torch.load('weights/bfw_senet50_logistic_regression_face_matching_0.0_betaTEST.pt')
# model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_loader = data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
val_loader = data.DataLoader(dataset=val_data, batch_size=BATCH_SIZE)
holdout_loader = data.DataLoader(dataset=holdout_data, batch_size=BATCH_SIZE)

In [9]:
def val_model(model, loader, criterion):
    """Validate model on loader with criterion function"""
    y_true, y_pred, y_prot = [], [], []
    model.eval()
    with torch.no_grad():
        for inputs, labels,protected in loader:
            inputs = inputs.to(device)
            y_true.append(labels)
            y_prot.append(protected)
            y_pred.append(torch.sigmoid(model(inputs)).cpu())
    y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
    return criterion(y_true, y_pred, y_prot)
def compute_bias(y_pred, y_true, prot, metric):
    """Compute bias on the dataset"""
    def zero_if_nan(data):
        """Zero if there is a nan"""
        return 0. if torch.isnan(data) else data

    gtpr_prot = zero_if_nan(y_pred[prot * y_true == 1].mean())
    gfpr_prot = zero_if_nan(y_pred[prot * (1-y_true) == 1].mean())
    mean_prot = zero_if_nan(y_pred[prot == 1].mean())

    gtpr_unprot = zero_if_nan(y_pred[(1-prot) * y_true == 1].mean())
    gfpr_unprot = zero_if_nan(y_pred[(1-prot) * (1-y_true) == 1].mean())
    mean_unprot = zero_if_nan(y_pred[(1-prot) == 1].mean())

    if metric == "spd":
        return mean_prot - mean_unprot
    elif metric == "aod":
        return 0.5 * ((gfpr_prot - gfpr_unprot) + (gtpr_prot - gtpr_unprot))
    elif metric == "eod":
        return gtpr_prot - gtpr_unprot
    elif 'false_diff':
        return (np.abs(gfpr_prot - gfpr_unprot) + np.abs(gtpr_unprot - gtpr_prot))

def compute_objective(performance, bias, epsilon=0.001, margin=0.0005):
    if abs(bias) <= (epsilon-margin):
        return performance
    else:
        return 0.0
def get_best_objective(y_true, y_pred, y_prot):
    """Find the threshold for the best objective"""
    num_samples = 5
    threshs = torch.linspace(0, 1, 501)
    best_obj, best_thresh = -math.inf, 0.
    for thresh in threshs:
        indices = np.random.choice(np.arange(y_pred.size()[0]), num_samples*y_pred.size()[0], replace=True).reshape(num_samples, y_pred.size()[0])
        objs = []
        for index in indices:
            y_pred_tmp = y_pred[index]
            y_true_tmp = y_true[index]
            y_prot_tmp = y_prot[index]
            perf = (torch.mean((y_pred_tmp > thresh)[y_true_tmp.type(torch.bool)].type(torch.float32)) + torch.mean((y_pred_tmp <= thresh)[~y_true_tmp.type(torch.bool)].type(torch.float32))) / 2
            bias = compute_bias((y_pred_tmp > thresh).float().cpu(), y_true_tmp.float().cpu(), y_prot_tmp.float().cpu(), 'false_diff')
            objs.append(compute_objective(perf, bias))
        obj = float(torch.tensor(objs).mean())
        if obj > best_obj:
            best_obj, best_thresh = obj, thresh

    return best_obj, best_thresh
class Critic(nn.Module):
    """Critic class for adversarial debiasing method"""

    def __init__(self, sizein, num_deep=3, hid=32):
        super().__init__()
        self.fc0 = nn.Linear(sizein, hid)
        self.fcs = nn.ModuleList([nn.Linear(hid, hid) for _ in range(num_deep)])
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(hid, 1)

    def forward(self, t):
        t = t.reshape(1, -1)
        t = self.fc0(t)
        for fully_connected in self.fcs:
            t = F.relu(fully_connected(t))
            t = self.dropout(t)
        return self.out(t)


In [10]:
# compute bias before applying random processing
y_true, y_pred, y_prot = [], [], []
# model.eval()
with torch.no_grad():
    for inputs, labels,protected in val_loader:
        inputs = inputs.to(device)
        y_true.append(labels)
        y_prot.append(protected)
        y_pred.append(torch.sigmoid(model(inputs)).cpu())
y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)

compute_bias((y_pred>0.5).float().cpu(), y_true, y_prot, 'false_diff')


tensor(0.0032)

In [11]:
# adversarial:
margin = 0.01
epochs = 10
critic_steps = 301
actor_steps = 401
lmbda = 20
checkpoint = 'adversarial_checkpoint.pt'

unrefined_net = BinaryClassification()
base_model = copy.deepcopy(unrefined_net)
base_model.layer_out = nn.Linear(base_model.layer_out.in_features, base_model.layer_out.in_features)

actor = nn.Sequential(base_model, nn.Linear(base_model.layer_out.in_features, 1))
actor.to(device)
actor_optimizer = optim.Adam(actor.parameters(), lr=1e-4)
actor_loss_fn = nn.BCEWithLogitsLoss()
actor_loss = 0.


critic = Critic(BATCH_SIZE*unrefined_net.layer_out.in_features)
critic.to(device)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-4)
critic_loss_fn = nn.MSELoss()
critic_loss = 0.
critic_steps = BATCH_SIZE

for epoch in range(epochs):
    for param in critic.parameters():
        param.requires_grad = True
    for param in actor.parameters():
        param.requires_grad = False
    actor.eval()
    critic.train()
    for step, (inputs, labels, prot) in enumerate(test_loader):
        if step > critic_steps:
            break
        inputs, labels = inputs.to(device), labels.to(device)
        if inputs.size(0) != BATCH_SIZE:
            continue
        critic_optimizer.zero_grad()

        with torch.no_grad():
            y_pred = actor(inputs)

        y_true = labels.float().to(device)
        y_prot = prot.float().to(device)

        bias = compute_bias((y_pred>0.5).float().cpu(), y_true.float().cpu(), y_prot.float().cpu(), 'false_diff').to(device)
        res = critic(base_model(inputs))
        loss = critic_loss_fn(bias.unsqueeze(0), res[0])
        loss.backward()
        critic_loss += loss.item()
        critic_optimizer.step()
        if step % 100 == 0:
            print_loss = critic_loss if (epoch*critic_steps + step) == 0 else critic_loss / (epoch*critic_steps + step)
            print(f'=======> Epoch: {(epoch, step)} Critic loss: {print_loss:.3f}')

    for param in critic.parameters():
        param.requires_grad = False
    for param in actor.parameters():
        param.requires_grad = True
    actor.train()
    critic.eval()
    for step, (inputs, labels,prot) in enumerate(test_loader):
        if step > actor_steps:
            break
        inputs, labels = inputs.to(device), labels.to(device)
        if inputs.size(0) != BATCH_SIZE:
            continue
        actor_optimizer.zero_grad()

        y_true = labels.float().to(device)
        y_prot = prot.float().to(device)

        est_bias = critic(base_model(inputs))
        loss = actor_loss_fn(actor(inputs)[:, 0], y_true)

        loss = max(1, lmbda*(abs(est_bias)-0.05+epochs)+1) * loss

        loss.backward()
        actor_loss += loss.item()
        actor_optimizer.step()
        if step % 100 == 0:
            print_loss = critic_loss if (epoch*actor_steps + step) == 0 else critic_loss / (epoch*actor_steps + step)
            print(f'=======> Epoch: {(epoch, step)} Actor loss: {print_loss:.3f}')


_, best_thresh = val_model(actor, test_loader, get_best_objective)




=======> Epoch: (0, 0) Critic loss: 0.005
=======> Epoch: (0, 0) Actor loss: 0.256
=======> Epoch: (0, 100) Actor loss: 0.003
=======> Epoch: (1, 0) Critic loss: 0.004
=======> Epoch: (1, 0) Actor loss: 0.002
=======> Epoch: (1, 100) Actor loss: 0.002
=======> Epoch: (2, 0) Critic loss: 0.007
=======> Epoch: (2, 0) Actor loss: 0.002
=======> Epoch: (2, 100) Actor loss: 0.002
=======> Epoch: (3, 0) Critic loss: 0.010
=======> Epoch: (3, 0) Actor loss: 0.002
=======> Epoch: (3, 100) Actor loss: 0.002
=======> Epoch: (4, 0) Critic loss: 0.011
=======> Epoch: (4, 0) Actor loss: 0.002
=======> Epoch: (4, 100) Actor loss: 0.002
=======> Epoch: (5, 0) Critic loss: 0.012
=======> Epoch: (5, 0) Actor loss: 0.003
=======> Epoch: (5, 100) Actor loss: 0.003
=======> Epoch: (6, 0) Critic loss: 0.015
=======> Epoch: (6, 0) Actor loss: 0.004
=======> Epoch: (6, 100) Actor loss: 0.004
=======> Epoch: (7, 0) Critic loss: 0.020
=======> Epoch: (7, 0) Actor loss: 0.004
=======> Epoch: (7, 100) Actor loss

In [12]:
y_true, y_pred, y_prot = [], [], []
actor.eval()
with torch.no_grad():
    for inputs, labels,protected in test_loader:
        inputs = inputs.to(device)
        y_true.append(labels)
        y_prot.append(protected)
        y_pred.append(torch.sigmoid(actor(inputs)).cpu())
y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
print(compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'false_diff'))
print(compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'aod'))

tensor(0.)
tensor(0.)


In [13]:
y_true, y_pred, y_prot = [], [], []
actor.eval()
with torch.no_grad():
    for inputs, labels,protected in holdout_loader:
        inputs = inputs.to(device)
        y_true.append(labels)
        y_prot.append(protected)
        y_pred.append(torch.sigmoid(actor(inputs)).cpu())
y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
print(compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'false_diff'))
print(compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'aod'))

tensor(0.)
tensor(0.)


In [14]:
torch.save(actor.state_dict(), 'weights/bfw_senet_adversarial_intraprocessing_best_model.pt')